In [1]:
import json
import pandas as pd
import re
import csv
import random
import numpy as np
from tqdm import tqdm

import spacy
import spacy.cli
import pt_core_news_md
from spacy.lang.pt import Portuguese
from spacy.pipeline import EntityRuler
from spacy.lang.pt.examples import sentences 
from spacy.tokens import DocBin
from spacy.training import offsets_to_biluo_tags

import sklearn as sk
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from matplotlib import pyplot
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
spacy.cli.download("pt_core_news_md")
nlp = spacy.load("pt_core_news_md")

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_md')


In [3]:
nlp = spacy.load(r'output/model-best')

In [5]:
def loadJSON(caminho):
    tf = open(caminho, "r")
    dicionario = json.load(tf)
    tf.close
    return(dicionario)

In [6]:
test_data = loadJSON(r'test_data.json')
test_data

[['na linha do disposto no artigo 39 da lei nº 4.886/65, a competência para julgar a ação em que se discutem direitos decorrentes de contrato de representação comercial é do foro do domicílio do representante.',
  {'entities': [[24, 52, 'FUNDAMENTO']]}],
 ['a peça, subscrita por advogado regularmente credenciado, foi protocolada no prazo assinado em lei.',
  {'entities': []}],
 ['há, ainda, petição apresentada pelo instituto brasileira de ética concorrencial – etco, entidade que figura como amicus curiae, na qual se manifesta sobre os embargos opostos pelo phs.',
  {'entities': [[36, 79, 'ORGANIZACAO'], [180, 183, 'ORGANIZACAO']]}],
 ['com efeito, seja pela possibilidade de repercussão da decisão estadual no âmbito federal, seja pelo cabimento de recurso extraordinário contra acórdão proferido no processo objetivo (re 190.985/sc, rel.',
  {'entities': []}],
 ['o senhor ministro marco aurélio – à folha 3.685 a 3.687, proferi decisão do seguinte teor: agravo - juízo de retratação – honor

In [8]:
dfTest_data = pd.DataFrame(test_data, columns=['textos', 'entidades'])
dfTest_data

,textos,entidades
0,na linha do disposto no artigo 39 da lei nº 4....,"{'entities': [[24, 52, 'FUNDAMENTO']]}"
1,"a peça, subscrita por advogado regularmente cr...",{'entities': []}
2,"há, ainda, petição apresentada pelo instituto ...","{'entities': [[36, 79, 'ORGANIZACAO'], [180, 1..."
3,"com efeito, seja pela possibilidade de repercu...",{'entities': []}
4,o senhor ministro marco aurélio – à folha 3.68...,"{'entities': [[2, 31, 'JUIZ']]}"
...,...,...
263,"tem-se nos autos que, em 7.12.2010, o paciente...","{'entities': [[63, 93, 'PESSOA'], [95, 117, 'P..."
264,4/5),{'entities': []}
265,116/117).,{'entities': []}
266,ausência do necessário prequestionamento.,{'entities': []}


In [19]:
def findEnt(texto: str, start: int, end: int):
    return texto[start:end]
    

In [21]:
listTest_data = []
for i in test_data:
    if i[1]['entities']:
        for j in i[1]['entities']:
            linha = [i[0], j[0], j[1], j[2], findEnt(i[0], j[0], j[1])]
            listTest_data.append(linha)
    else:
        linha = [i[0], 0, 0, ""]
        listTest_data.append(linha)
dfTest_data = pd.DataFrame(listTest_data, columns=['texto', 'start', 'end', 'rotulo', 'entidade'])

In [22]:
dfTest_data

,texto,start,end,rotulo,entidade
0,na linha do disposto no artigo 39 da lei nº 4....,24,52,FUNDAMENTO,artigo 39 da lei nº 4.886/65
1,"a peça, subscrita por advogado regularmente cr...",0,0,,None
2,"há, ainda, petição apresentada pelo instituto ...",36,79,ORGANIZACAO,instituto brasileira de ética concorrencial
3,"há, ainda, petição apresentada pelo instituto ...",180,183,ORGANIZACAO,phs
4,"com efeito, seja pela possibilidade de repercu...",0,0,,None
...,...,...,...,...,...
344,"tem-se nos autos que, em 7.12.2010, o paciente...",120,138,PESSOA,ana karla de sousa
345,4/5),0,0,,None
346,116/117).,0,0,,None
347,ausência do necessário prequestionamento.,0,0,,None


In [23]:
dfTest_data = pd.DataFrame(test_data, columns=['textos', 'true'])
for dfTest_data

,textos,entidades
0,na linha do disposto no artigo 39 da lei nº 4....,"{'entities': [[24, 52, 'FUNDAMENTO']]}"
1,"a peça, subscrita por advogado regularmente cr...",{'entities': []}
2,"há, ainda, petição apresentada pelo instituto ...","{'entities': [[36, 79, 'ORGANIZACAO'], [180, 1..."
3,"com efeito, seja pela possibilidade de repercu...",{'entities': []}
4,o senhor ministro marco aurélio – à folha 3.68...,"{'entities': [[2, 31, 'JUIZ']]}"
...,...,...
263,"tem-se nos autos que, em 7.12.2010, o paciente...","{'entities': [[63, 93, 'PESSOA'], [95, 117, 'P..."
264,4/5),{'entities': []}
265,116/117).,{'entities': []}
266,ausência do necessário prequestionamento.,{'entities': []}


In [37]:
listTest_data = []
for i in dfTest_data.index:
    tokens = nlp(dfTest_data['textos'].iloc[i])
    entities = []
    for token in tokens.ents:
        if token.ents:
            entities.append([str(token.label_), token.start_char, token.end_char, str(token)])
        else:
            entities.append([])
    listTest_data.append(entities)
listTest_data


[[['FUNDAMENTO', 24, 52, 'artigo 39 da lei nº 4.886/65']],
 [],
 [['ORGANIZACAO', 36, 79, 'instituto brasileira de ética concorrencial'],
  ['ORGANIZACAO', 180, 183, 'phs']],
 [['FUNDAMENTO', 183, 202, 're 190.985/sc, rel.']],
 [['JUIZ', 2, 31, 'senhor ministro marco aurélio']],
 [],
 [],
 [],
 [['TRIBUNAL', 2, 42, 'tribunal de justiça do rio grande do sul'],
  ['ORGANIZACAO', 88, 106, 'ministério público']],
 [],
 [],
 [],
 [['ORGANIZACAO', 40, 55, 'polícia federal']],
 [],
 [['FUNDAMENTO', 141, 163, 'lei municipal 772/2004']],
 [],
 [['FUNDAMENTO', 49, 96, 'artigo 170, inciso iii, da constituição federal']],
 [],
 [['FUNDAMENTO', 30, 57, 'sl n. 292/pe, dje 14.4.2009'],
  ['FUNDAMENTO', 59, 86, 'sl n. 287/sp, dje 18.5.2009'],
  ['FUNDAMENTO', 90, 116, 'sl n. 286/mg, dje 5.8.2009']],
 [],
 [],
 [['FUNDAMENTO', 41, 55, 'art. 544, § 4º']],
 [['FUNDAMENTO', 77, 94, 're nº 572.884-agr']],
 [],
 [],
 [],
 [],
 [['TRIBUNAL', 143, 150, 'supremo']],
 [],
 [['FUNDAMENTO', 55, 62, 'art. 93']],
 

In [36]:
len(listTest_data)

268

In [38]:
dfTest_data['predict'] = listTest_data
dfTest_data

,textos,entidades,predict
0,na linha do disposto no artigo 39 da lei nº 4....,"{'entities': [[24, 52, 'FUNDAMENTO']]}","[[FUNDAMENTO, 24, 52, artigo 39 da lei nº 4.88..."
1,"a peça, subscrita por advogado regularmente cr...",{'entities': []},[]
2,"há, ainda, petição apresentada pelo instituto ...","{'entities': [[36, 79, 'ORGANIZACAO'], [180, 1...","[[ORGANIZACAO, 36, 79, instituto brasileira de..."
3,"com efeito, seja pela possibilidade de repercu...",{'entities': []},"[[FUNDAMENTO, 183, 202, re 190.985/sc, rel.]]"
4,o senhor ministro marco aurélio – à folha 3.68...,"{'entities': [[2, 31, 'JUIZ']]}","[[JUIZ, 2, 31, senhor ministro marco aurélio]]"
...,...,...,...
263,"tem-se nos autos que, em 7.12.2010, o paciente...","{'entities': [[63, 93, 'PESSOA'], [95, 117, 'P...","[[PESSOA, 63, 93, alípio vicente ferreira júni..."
264,4/5),{'entities': []},[]
265,116/117).,{'entities': []},[]
266,ausência do necessário prequestionamento.,{'entities': []},[]


In [39]:
dfTest_data.to_excel('dfConferencia.xlsx')